In [1]:
import pandas as pd
import re
from utils import valid_params, load_properties,convert_to_output

## Leitura e tratamento dos dados

In [2]:
df_imoveis = load_properties()
len(df_imoveis)

5079

## Base utilizada

In [3]:
df_imoveis.head(3)

,id,titulo,value,location,price_suggested,nearby_locations,categoria,area,rooms,bathrooms,garages,district_name
0,607458fa77dee513e95c7f1c,Parque Alameda Real - Apartamento 2 quartos em...,138900.0,"{'lat': -10.9472468, 'lng': -37.0730823}",147871.0,18,Apartamentos,40.0,3.0,2.0,1.0,Santa Maria
1,607458ff77dee513e95c7f1d,::Ecoville park,330000.0,"{'lat': -10.9396559, 'lng': -37.0593199}",367439.0,23,Apartamentos,62.0,3.0,2.0,1.0,Jardins
2,6074590377dee513e95c7f1e,::Ecoville park,330000.0,"{'lat': -10.9396559, 'lng': -37.0593199}",367439.0,23,Apartamentos,62.0,3.0,2.0,1.0,Jardins


## Bairros válidos

In [4]:
AVAILABLE_DISTRICTS = ['Jabotiana', 'Atalaia', 'Salgado Filho', 'Lamarão', 'Grageru',
                       'Luzia', 'Jardins', 'Farolândia', 'Coroa do Meio', 'Ponto Novo',
                       'Suíssa', 'Centro', 'São Conrado', 'Inácio Barbosa', 'São José',
                       'Zona de Expansão (Mosqueiro)', 'Treze de Julho',
                       'Dezoito do Forte', 'Aeroporto', 'Zona de Expansão (Aruana)',
                       'Siqueira Campos', 'Pereira Lobo', 'Jardim Centenário',
                       'Santa Maria', 'Dom Luciano', 'Olaria', 'Santos Dumont', 'América',
                       'Santo Antônio', 'Getúlio Vargas', 'Cirurgia', 'Industrial',
                       "Porto D'Antas", '18 do Forte', 'José Conrado de Araújo',
                       'Cidade Nova', 'Zona de Expansão (Robalo)', '13 De Julho',
                       'Palestina', 'Soledade', 'Japãozinho']

## Preferências do Usuário 

In [9]:
params = {
    "valor": 300000,        # Valor em Reais (R$)
    "quartos": 3,             # Número de quartos
    "area": 80,             # Área do imóvel em metros quadrados
    "banheiros": 2,         # Número de banheiros do imóvel
    "garagens": 1,           # Número de garagens do imóvel
    "bairro": "Atalaia",  # Bairro onde se localiza o imóvel
    "latitude": -10.995901, # latitude desejada
    "longitude": -37.055213 # longitude desejada
}

## Validar parametros 

In [10]:
from utils import valid_params

error = valid_params(params)
if error != None:
    print(error)

value is a required field


## Preparar os parametros

In [11]:
from utils import prepare_params

prepared_params, error = prepare_params(params)

In [12]:
prepared_params

{'value': 300000.0,
 'garages': 1.0,
 'area': 80.0,
 'bathrooms': 2.0,
 'district': 'Atalaia',
 'location': {'lat': -10.995901, 'lng': -37.055213}}

## Alterar a base de acordo com os parametros

In [9]:
from utils import handle_dataframe_values

df_to_algorithm, params_to_algorithm = handle_dataframe_values(df_imoveis, prepared_params)
df_to_algorithm.head(3)

,id,titulo,value,location,price_suggested,nearby_locations,categoria,area,rooms,bathrooms,garages,district_name,distance,district
0,607458fa77dee513e95c7f1c,Parque Alameda Real - Apartamento 2 quartos em...,138900.0,"{'lat': -10.9472468, 'lng': -37.0730823}",147871.0,18,Apartamentos,40.0,3.0,2.0,1.0,Santa Maria,4.999925,0.0
1,607458ff77dee513e95c7f1d,::Ecoville park,330000.0,"{'lat': -10.9396559, 'lng': -37.0593199}",367439.0,23,Apartamentos,62.0,3.0,2.0,1.0,Jardins,5.394045,0.0
2,6074590377dee513e95c7f1e,::Ecoville park,330000.0,"{'lat': -10.9396559, 'lng': -37.0593199}",367439.0,23,Apartamentos,62.0,3.0,2.0,1.0,Jardins,5.394045,0.0


## Definição dos pesos de cada caracteristica

In [10]:
weights = {
    'value': [30.0],
    'rooms': [10.0],
    'area': [10.0],
    'bathrooms': [10.0],
    'garages': [10.0],
    'district': [10.0],
    'distance': [10.0]
}

## Aplicando Algoritmo de Recomendação

In [11]:
from utils import mahalanobis_algorithm

sorted_values = mahalanobis_algorithm(df_to_algorithm, params_to_algorithm, weights)

result = {}
for index in range(3):
    aux = df_to_algorithm.iloc[sorted_values.iloc[index].name].to_dict()
    aux['similarity'] = sorted_values.iloc[index][0]
    aux['similarity_index'] = int(sorted_values.iloc[index].name)

    result.update({index: convert_to_output(aux)})
    df_to_algorithm = df_to_algorithm[df_to_algorithm.id != aux['id']]
    
pd.DataFrame.from_dict(result, orient="index")

,id,titulo,area,banheiros,bairro,preco_estimado,similaridade,valor,quartos,garagens,locais_proximos,distancia
0,607f33ac4c5fd7aeae9a73c5,EDF Pontal do Atlântico de; 3/4,80.0,2.0,Atalaia,327663.0,2.833835,300000.0,3.0,1.0,15,0.482151
1,6085cb97016b09b70c75bc23,"Apartamento com 3 dormitórios à venda, 124 m² ...",124.0,2.0,Salgado Filho,312255.0,3.066160,450000.0,3.0,1.0,19,6.471091
2,607e8a51092e9f82c3b1eba2,285.000,77.0,2.0,Atalaia,321778.0,3.590341,285000.0,3.0,1.0,14,0.470860
